<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Load-stuff" data-toc-modified-id="Load-stuff-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load stuff</a></span><ul class="toc-item"><li><span><a href="#Packages" data-toc-modified-id="Packages-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Packages</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Paths</a></span></li><li><span><a href="#Preprocessor" data-toc-modified-id="Preprocessor-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Preprocessor</a></span></li></ul></li><li><span><a href="#XGBoost-model-training" data-toc-modified-id="XGBoost-model-training-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>XGBoost model training</a></span></li></ul></div>

## Load stuff

### Packages

In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
from tqdm import tqdm, trange, tnrange, tqdm_notebook
import os

### Paths

In [2]:
%%time
from HERE import PROJ_ROOT_DIR, DOWNLOADS_DIR, SCRATCH_DIR
TRAIN_CSV_PATH = os.path.join(DOWNLOADS_DIR, 'train.csv')
TRAIN_N_SAMPLES = sum(1 for row in open(TRAIN_CSV_PATH, 'r'))-1
print(TRAIN_N_SAMPLES, 'samples in', TRAIN_CSV_PATH)

184903890 samples in D:\Dropbox\Projects\Kaggle-TalkingData-AdTracking-Fraud-Detection\downloads\train.csv
Wall time: 36.6 s


### Preprocessor

In [3]:
from preprocess import Preprocessor, ORIGINAL_FEATURES, CAT_COLS, NUM_COLS, TARGET_COL
prep = Preprocessor()

## XGBoost model training

Training parameters

In [ ]:
CHUNK_SIZE = 100000
N_CHUNKS = TRAIN_N_SAMPLES // CHUNK_SIZE + 1
N_EPOCHS = 5  # number of times to go over the data
try_xgb_params = {'colsample_bytree': 0.4,
                  'eta': 0.1,
                  'gamma': 2,
                  'max_depth': 4,
                  'min_child_weight': 100,
                  'objective': 'reg:linear',
                  'seed': 2018,
                  'subsample': 1}

In [ ]:
%%time
xgb_model = None
for epoch in tnrange(N_EPOCHS, desc='Epoch', ascii=True, unit='epoch', position=0):
    for train_chunk in tqdm_notebook(pd.read_csv(os.path.join(DOWNLOADS_DIR, 'train.csv'), chunksize=CHUNK_SIZE), 
                            desc='Chunk', ascii=True, position=1, total=N_CHUNKS, leave=False, unit='chunk'):
        data_matrix = xgb.DMatrix(prep.transform(train_chunk), train_chunk[TARGET_COL])
        xgb_model = xgb.train(params=try_xgb_params, dtrain=data_matrix, xgb_model=xgb_model)